In [ ]:
from VQE.Methods import ADAPT_minimization, ADAPT_mixed_minimization
from VQE.Circuit import Circuits_Composser, Qibo_measure_Energy
from qibo import  set_backend

import contextlib
import io

import numpy as np
from tqdm import tqdm

set_backend("qibojit")

f = io.StringIO()


[Qibo 0.2.18|INFO|2025-06-11 13:46:12]: Using qibojit (numba) backend on /CPU:0


In [2]:
n_qubits = 6
nuc='Be6'
ref_state=0
data, nucleo = ADAPT_minimization(nuc, ref_state = ref_state, n_qubits=n_qubits, max_layers=3)

 --------------------------------------------------------------------------
                            ADAPT for  Be6
 --------------------------------------------------------------------------

Initial Energy:  1.5679
[np.float64(0.5781652654111846)]

------------ LAYER 1 ------------
Energy:  -0.8882113101561737
Rel. Error:  0.7086683621340858
New Operator:  [0, 3, 4, 5]     Theta: 0.5781652654111846

------------ LAYER 2 ------------
Energy:  -3.0487955892755996
Rel. Error:  7.854146083271598e-07
New operator:  [0, 3, 1, 2]     Theta: -0.7861368094572897

Operators used for each layer:
Layer 0: Operator [0, 3, 4, 5], Theta = 0.5913989227889727, Gradient = 7.527717371155747
Layer 1: Operator [0, 3, 1, 2], Theta = -0.7861368094572897, Gradient = 4.908336128743321

 Ground state aproximation:
[(0, np.float64(0.5865791138377705)), (1, np.float64(-0.587446303243964)), (4, np.float64(0.5575228999909301))]

 Final energy result: -3.0487955892755996	 Final relative error is 7.8541460832715

In [3]:

used_operators = data['used_operators']
thetas = data['parameters']
ham_pool = data['ham_pool']
monoparticular_energies = data['monoparticular']
two_index = data['two_index']

composer = Circuits_Composser(operator_pool=ham_pool, 
                              operators_used=used_operators,
                              parameters=thetas, 
                              name=nuc, 
                              n_qubits=n_qubits, 
                              ref_state=ref_state, 
                              only_ref=False) 


Qibo_circuits = composer.Qibo_all_circuits()

print(len(Qibo_circuits)) 

13


In [4]:
Et_exact = Qibo_measure_Energy(monoparticular_energies, two_index, Qibo_circuits, exact=True)

(0.55752+0j)|000011> + (-0.58745+0j)|011000> + (0.58658+0j)|100100> 

Energy,			observable,		amplitude

0.56084234 		 0 		 1.63
0.56250185 		 1 		 1.63
0.56250185 		 2 		 1.63
0.56084234 		 3 		 1.63
0.70558815 		 4 		 2.27
0.70558815 		 5 		 2.27
-0.5822094 		 [0, 3, 0, 3] 		 -1.6921
-0.58393213 		 [1, 2, 1, 2] 		 -1.6921
0.10558956 		 [4, 5, 4, 5] 		 0.3397
-0.71887058 		 [0, 3, 1, 2] 		 1.0431
-2.46179911 		 [0, 3, 4, 5] 		 -3.76385869
-2.4654386 		 [1, 2, 4, 5] 		 3.76385869

-----------------------------------Final Energies-----------------------------------

Monoparticular energies:                  3.657864683538295
Two body energies with 2 diferent index:  -1.060551981239751
Two body energies with 3 diferent index:  0.0
Two body energies with 4 diferent index:  -5.646108291574137


In [5]:
Et_shots = Qibo_measure_Energy(monoparticular_energies, two_index, Qibo_circuits, exact=False, nshots=1000)

Energy,			observable,		amplitude

0.53953 		 0 		 1.63
0.57702 		 1 		 1.63
0.57702 		 2 		 1.63
0.53953 		 3 		 1.63
0.71505 		 4 		 2.27
0.71505 		 5 		 2.27
-0.5600851 		 [0, 3, 0, 3] 		 -1.6921
-0.5990034 		 [1, 2, 1, 2] 		 -1.6921
0.1070055 		 [4, 5, 4, 5] 		 0.3397
-0.7207821 		 [0, 3, 1, 2] 		 1.0431
0.0022055 		 [0, 3, 1, 5] 		 -1.10275
0.00110275 		 [0, 3, 2, 4] 		 -1.10275
-2.44274429 		 [0, 3, 4, 5] 		 -3.76385869
-2.43898043 		 [1, 2, 4, 5] 		 3.76385869
-0.00771925 		 [2, 1, 4] 		 -1.10275
-0.02756875 		 [1, 2, 5] 		 -1.10275

-----------------------------------Final Energies-----------------------------------

Monoparticular energies:                  3.663199999999999
Two body energies with 2 diferent index:  -1.0520829999999997
Two body energies with 3 diferent index:  -0.03528800000000003
Two body energies with 4 diferent index:  -5.599198565194502


# Pruebas

In [6]:

list_shots=[100, 300, 1000, 3000, 10000]

desviaciones_fixed=[]
medias_fixed=[]
for shots in list_shots:
    energies=[]
    print('----------------',shots,'----------------')
    for _ in tqdm(range(200), leave=False, desc="Repeticiones"):
        with contextlib.redirect_stdout(f):
            Et_shots = Qibo_measure_Energy(monoparticular_energies, two_index, Qibo_circuits, exact=False, nshots=shots)
        energies.append(Et_shots)
    desviaciones_fixed.append(np.std(energies))
    medias_fixed.append(np.mean(energies))
    print('DONE')
    
    

---------------- 100 ----------------


DONE
---------------- 300 ----------------


DONE
---------------- 1000 ----------------


DONE
---------------- 3000 ----------------


DONE
---------------- 10000 ----------------


DONE


In [7]:
data_mixed, nucleo=ADAPT_mixed_minimization(data=data, nucleus=nuc, ref_state = ref_state, n_qubits=n_qubits, max_layers=5, exact=False, opt_method='COBYLA', nshots=1000)

0
Energy,			observable,		amplitude

1.63 		 0 		 1.63
1.63 		 3 		 1.63
-1.6921 		 [0, 3, 0, 3] 		 -1.6921
-0.0187758 		 [0, 3, 1, 2] 		 1.0431
0.0374935 		 [0, 3, 1, 5] 		 -1.10275
0.0198495 		 [0, 3, 2, 4] 		 -1.10275
-0.06774946 		 [0, 3, 4, 5] 		 -3.76385869

-----------------------------------Final Energies-----------------------------------

Monoparticular energies:                  3.26
Two body energies with 2 diferent index:  -1.6920999999999997
Two body energies with 3 diferent index:  0.0
Two body energies with 4 diferent index:  -0.02918225634040178
 --------------------------------------------------------------------------
                            ADAPT for  Be6
 --------------------------------------------------------------------------

0
Energy,			observable,		amplitude

1.63 		 0 		 1.63
1.63 		 3 		 1.63
-1.6921 		 [0, 3, 0, 3] 		 -1.6921
-0.0542412 		 [0, 3, 1, 2] 		 1.0431
0.0286715 		 [0, 3, 1, 5] 		 -1.10275
-0.0066165 		 [0, 3, 2, 4] 		 -1.10275
-0.08280489 		 

In [8]:
nshots=[100, 300, 1000, 3000, 10000]
results={}
k=0
with open("Be6_2_layer.dat", "w") as file:
    file.write("shots theta1_mean theta1_std theta2_mean theta2_std energy_mean energy_std\n")
    
for shots in nshots:
    fixed_E=medias_fixed[k]
    fiexd_std=desviaciones_fixed[k]
    thetas_1=[]
    thetas_2=[]
    thetas_3=[]
    energias=[]
    print('--------------', shots, ' shots --------------')
    for _ in tqdm(range(200), leave=False, desc="Repeticiones"):
        with contextlib.redirect_stdout(f):       
            data_mixed, nucleo=ADAPT_mixed_minimization(data=data, nucleus=nuc, ref_state = ref_state, n_qubits=n_qubits, max_layers=5, exact=False, opt_method='COBYLA', nshots=shots)
        thetas_1.append(data_mixed['parameters'][0])
        thetas_2.append(data_mixed['parameters'][1])

        energias.append(data_mixed['Energy'])
        
    print('DONE')
    results[shots] = {'theta_1': [np.mean(thetas_1), np.std(thetas_1)], 'theta_2':[np.mean(thetas_2), np.std(thetas_2)],  'energy':[np.mean(energias), np.std(energias)]}
    
    media_t1, std_t1 = results[shots]['theta_1']
    media_t2, std_t2 = results[shots]['theta_2']

    media_E, std_E = results[shots]['energy']
    
    with open("Be6_2_layer.dat", "a") as file:
        file.write(f"{shots} {media_t1:.6f} {std_t1:.6f} {media_t2:.6f} {std_t2:.6f} {media_E:.6f} {std_E:.6f} {fixed_E:.6f} {fiexd_std:.6f}\n")


-------------- 100  shots --------------


DONE
-------------- 300  shots --------------


DONE
-------------- 1000  shots --------------


DONE
-------------- 3000  shots --------------


DONE
-------------- 10000  shots --------------


DONE
